<a href="https://colab.research.google.com/github/Atfssene/FRASA/blob/main/Text_Summarization_Model_FRASA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text Summarization Model

In this notebook, we will create a model for text summarization task. TextRank and SumBasic will be our feature extraction from senteces to create a weights that will be feeded to a neural networks. Let's start!

## Import library

In [1]:
import nltk
nltk.download('punkt')
# !pip install fasttext

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [50]:
from nltk.tokenize import sent_tokenize, word_tokenize
import numpy as np
# import gzip
import networkx as nx
import tensorflow as tf
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
# import fasttext
# import fasttext.util

## Read data

In [3]:
train = tf.keras.utils.get_file('train.csv', 'https://raw.githubusercontent.com/Atfssene/FRASA/main/Text%20Summarization/train.csv')
test = tf.keras.utils.get_file('test.csv', 'https://raw.githubusercontent.com/Atfssene/FRASA/main/Text%20Summarization/test.csv')

df_train = pd.read_csv(train)
df_test = pd.read_csv(test)
df_train.info()
df_test.info()

9879552/9872406 [==============================] - 0s 0us/step
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15012 entries, 0 to 15011
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   labels      15012 non-null  object
 1   paragraphs  15012 non-null  object
 2   summary     15012 non-null  object
 3   clean_text  15012 non-null  object
dtypes: object(4)
memory usage: 469.2+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3762 entries, 0 to 3761
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   labels      3762 non-null   object
 1   paragraphs  3762 non-null   object
 2   summary     3762 non-null   object
dtypes: object(3)
memory usage: 88.3+ KB


## Create TextRank

Download Indonesian word vector

In [4]:
# !wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.id.300.vec.gz
# !wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.id.300.bin.gz

Unzipping...

In [5]:
# !gunzip cc.id.300.vec.gz
# !gunzip cc.id.300.bin.gz
# from gensim.models.keyedvectors import KeyedVectors
# kv = KeyedVectors.load_word2vec_format('/content/drive/MyDrive/model_summarization/cc.id.300.vec', limit=400000)

In [6]:
# kv.save_word2vec_format("/content/cc.id.vec", binary=False)

Load pretrained words embeddings

In [7]:
word_embeddings = {}
f = open('/content/drive/MyDrive/model_summarization/cc.id.vec', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

len(word_embeddings)

400001

In [26]:
sentences = []
for s in df_train['paragraphs'].head(1):
  sentences.append(sent_tokenize(s))
sentences = [y for x in sentences for y in x]

clean_sentences = []
for s in df_train['clean_text'].head(1):
  clean_sentences.append(sent_tokenize(s))
clean_sentences = [y for x in clean_sentences for y in x]

In [27]:
sentence_vectors = []
for i in clean_sentences:
  if len(i) != 0:
    v = sum([word_embeddings.get(w, np.zeros((300,))) for w in i.split()])/(len(i.split())+0.001)
  else:
    v = np.zeros((300,))
  sentence_vectors.append(v)

In [28]:
sim_mat = np.zeros([len(sentences), len(sentences)])

In [29]:
for i in range(len(sentences)):
  for j in range(len(sentences)):
    if i != j:
      sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,300), sentence_vectors[j].reshape(1,300))[0,0]

In [45]:
nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)

In [86]:
# [(i, s) for i,s in enumerate(sentences)]
for text in sentences:
  print(text)

Jakarta, CNN Indonesia - - Dokter Ryan Thamrin, yang terkenal lewat acara Dokter Oz Indonesia, meninggal dunia pada Jumat (4 / 8) dini hari.
Dokter Lula Kamal yang merupakan selebriti sekaligus rekan kerja Ryan menyebut kawannya itu sudah sakit sejak setahun yang lalu.
Lula menuturkan, sakit itu membuat Ryan mesti vakum dari semua kegiatannya, termasuk menjadi pembawa acara Dokter Oz Indonesia.
Kondisi itu membuat Ryan harus kembali ke kampung halamannya di Pekanbaru, Riau untuk menjalani istirahat. "
Setahu saya dia orangnya sehat, tapi tahun lalu saya dengar dia sakit.
( Karena) sakitnya, ia langsung pulang ke Pekanbaru, jadi kami yang mau jenguk juga susah.
Barangkali mau istirahat, ya betul juga, kalau di Jakarta susah isirahatnya, " kata Lula kepada CNNIndonesia.com, Jumat (4 / 8).
Lula yang mengenal Ryan sejak sebelum aktif berkarier di televisi mengaku belum sempat membesuk Ryan lantaran lokasi yang jauh.
Dia juga tak tahu penyakit apa yang diderita Ryan. "
Itu saya enggak tahu,

In [48]:
# for i in range(3):
#   print(ranked_sentences[i][1])
for i in ranked_sentences:
  print(i)

(0.06091013588314788, 'Lula menuturkan, sakit itu membuat Ryan mesti vakum dari semua kegiatannya, termasuk menjadi pembawa acara Dokter Oz Indonesia.')
(0.05986087469041391, 'Dokter Lula Kamal yang merupakan selebriti sekaligus rekan kerja Ryan menyebut kawannya itu sudah sakit sejak setahun yang lalu.')
(0.05837850794448731, 'Setahu saya dia orangnya sehat, tapi tahun lalu saya dengar dia sakit.')
(0.05811819645865672, '“ Saya tidak tahu, barangkali penyakit yang dulu sama yang sekarang berbeda, atau penyebab kematiannya beda dari penyakit sebelumnya.')
(0.05776311916576284, 'Meski demikian, ia mendengar beberapa kabar yang menyebut bahwa penyebab Ryan meninggal adalah karena jatuh di kamar mandi.')
(0.05773574513429258, 'Lula yang mengenal Ryan sejak sebelum aktif berkarier di televisi mengaku belum sempat membesuk Ryan lantaran lokasi yang jauh.')
(0.05656949054199408, 'Jakarta, CNN Indonesia - - Dokter Ryan Thamrin, yang terkenal lewat acara Dokter Oz Indonesia, meninggal dunia pa

## Create SumBasic

In [135]:
for sentence in (sent_tokenize(processed)):
  for word in word_tokenize(sentence):
    print(word)
    break

jakarta
dokter
lula
kondisi
setahu
sakitnya
barangkali
lula
tahu
enggak
enggak
setahun
meski
tahu
enggak
ryan
ryan
kemudian


In [161]:
frequency = {}
processed =  df_train['clean_text'].iloc[0]
for word in word_tokenize(processed):
  if word.isalnum():
    if word not in frequency.keys():
      frequency[word]=1
    else:
      frequency[word]+=1
max_fre = max(frequency.values())
for word in frequency.keys():
    frequency[word]=(frequency[word]/max_fre)
    
scores = {}
for i, sentence in enumerate((sent_tokenize(processed))):
  
  for word in word_tokenize(sentence):
    
    if word in frequency.keys():
        if i not in scores.keys():
          scores[i] = frequency[word]
        else:
          scores[i] += frequency[word]
scores

{0: 4.727272727272726,
 1: 3.909090909090908,
 2: 4.09090909090909,
 3: 1.9999999999999998,
 4: 1.0909090909090908,
 5: 0.9090909090909092,
 6: 2.0,
 7: 3.7272727272727266,
 8: 1.8181818181818183,
 9: 1.6363636363636362,
 10: 2.090909090909091,
 11: 2.0909090909090904,
 12: 2.2727272727272725,
 13: 2.181818181818181,
 14: 1.2727272727272727,
 15: 3.9090909090909083,
 16: 2.545454545454545,
 17: 1.636363636363636}

In [162]:
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)

In [164]:
for i in ranked_sentences:
  print(i)

(4.727272727272726, 'Jakarta, CNN Indonesia - - Dokter Ryan Thamrin, yang terkenal lewat acara Dokter Oz Indonesia, meninggal dunia pada Jumat (4 / 8) dini hari.')
(4.09090909090909, 'Lula menuturkan, sakit itu membuat Ryan mesti vakum dari semua kegiatannya, termasuk menjadi pembawa acara Dokter Oz Indonesia.')
(3.9090909090909083, 'Ryan Thamrin terkenal sebagai dokter yang rutin membagikan tips dan informasi kesehatan lewat tayangan Dokter Oz Indonesia.')
(3.909090909090908, 'Dokter Lula Kamal yang merupakan selebriti sekaligus rekan kerja Ryan menyebut kawannya itu sudah sakit sejak setahun yang lalu.')
(3.7272727272727266, 'Lula yang mengenal Ryan sejak sebelum aktif berkarier di televisi mengaku belum sempat membesuk Ryan lantaran lokasi yang jauh.')
(2.545454545454545, 'Ryan menempuh Pendidikan Dokter pada tahun 2002 di Fakultas Kedokteran Universitas Gadjah Mada.')
(2.2727272727272725, 'Meski demikian, ia mendengar beberapa kabar yang menyebut bahwa penyebab Ryan meninggal adala

## Neural Network